# Import

In [1]:
import numpy as np
import pandas as pd
import csv
import datetime

# myLibrary

In [2]:
#製造answer.csv的第1列
def answer_header():
    header = ['user_id']
    for i in range(28):
        header.append('time_slot_' + str(i) )
    return header

In [3]:
def get_week(date_str):
    d = datetime.datetime.strptime(date_str, '%Y-%m-%d')
    w = datetime.datetime.strftime(d, '%w')
    return int(w)

In [4]:
def get_slot(time_str):
    the_time = datetime.datetime.strptime(time_str, '%H:%M:%S.%f')
    time_sec0 = datetime.datetime.strptime('01:00:00.00', '%H:%M:%S.%f')
    time_sec1 = datetime.datetime.strptime('09:00:00.00', '%H:%M:%S.%f')
    time_sec2 = datetime.datetime.strptime('17:00:00.00', '%H:%M:%S.%f')
    time_sec3 = datetime.datetime.strptime('21:00:00.00', '%H:%M:%S.%f')
    time_sec4 = datetime.datetime.strptime('23:59:59.99', '%H:%M:%S.%f')
    if time_sec0 <= the_time and the_time <= time_sec1:
        slot = 0
    elif time_sec1 <= the_time and the_time <= time_sec2:
        slot = 1
    elif time_sec2 <= the_time and the_time <= time_sec3:
        slot = 2
    elif time_sec3 <= the_time and the_time <= time_sec4:
        slot = 3
    else:
        slot = 4
    return slot

In [5]:
def update_week(week, event_week, addition):
    week.days[event_week].add_slots(addition)
    another_addition = [0, 0, 0]
    another_addition.append(addition[4])
    week.days[event_week-1].add_slots(another_addition)
    return week

# myClass

In [6]:
class Day():
    def __init__(self):
        self.slot0, self.slot1, self.slot2, self.slot3 = 0, 0, 0, 0
    def reset(self):
        self.slot0, self.slot1, self.slot2, self.slot3 = 0, 0, 0, 0
    def get_slots(self):
        return [self.slot0, self.slot1, self.slot2, self.slot3]
    def add_slots(self, addition):
        self.slot0 += addition[0]
        self.slot1 += addition[1]
        self.slot2 += addition[2]
        self.slot3 += addition[3]

In [7]:
class Week():
    def __init__(self):
        self.days = []
        for i in range(7):
            self.days.append(Day())
    def reset(self):
        for i in range(7):
            self.days[i].reset()
    def get_answer(self, id):
        tmp = []
        for i in range(1, 7):
            tmp += self.days[i].get_slots()
        tmp += self.days[0].get_slots()
        answer = [id] + list(np.array(tmp)/32)
        return answer

# Main

In [8]:
with open('answer.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(answer_header())
    week = Week()
    # 開始處理資料
    for data_count in range(46, 76): # revise
        data = pd.read_csv('private/data-0' + str(data_count) + '.csv')
        
        # 取得目前csv檔案的首尾user_id
        head = data.head(1)['user_id'].values[0]
        tail = data.tail(1)['user_id'].values[0]
        
        # 依序處理各id資料並寫入answer.csv
        for id in range(head, tail+1):
            id_data = data[data['user_id'] == id]
            length = len(id_data)
            
            # 依序處理某id的每一筆資料
            now_date = id_data['event_time'].values[0].split()[0]
            addition = [0, 0, 0, 0, 0]
            for i in range(length):
                # 切分時間獲得星期和slot
                event_time = id_data['event_time'].values[i].split()
                date = event_time[0]
                
                # 當日期變化
                if date != now_date:
                    # 將前日的紀錄更新進week
                    event_week = get_week(now_date)
                    week = update_week(week, event_week, addition)
                    #並初始化now_date和addition
                    now_date = date
                    addition = [0, 0, 0, 0, 0]
                
                # 紀錄活動至addition
                event_slot = get_slot(event_time[1])
                addition[event_slot] = 1
            
            # 補齊最後一段日期的紀錄
            event_week = get_week(now_date)
            week = update_week(week, event_week, addition)
            
            # 寫入answer.csv
            writer.writerow(week.get_answer(id))
            
            # 初始化week
            week.reset()
            
            #break # revise

# test

# Test Area

In [9]:
id_data = data[data['user_id'] == 57160]
len(id_data)

0

In [10]:
data

,user_id,device_id,session_id,title_id,event_time,played_duration,action_trigger,platform,episode_number,series_total_episodes_count,internet_connection_type,is_trailer
0,93913,188172,9206492,16,2017-01-05 14:01:01.248,2,interrupt,iOS,16,17,wifi,False
1,93913,188172,9206492,16,2017-01-05 14:02:06.008,55,leave,iOS,17,17,wifi,False
2,93913,188172,9206492,16,2017-01-05 14:19:23.126,1029,interrupt,iOS,16,17,wifi,False
3,93913,188172,9206492,16,2017-01-05 15:02:57.054,2587,leave,iOS,16,17,wifi,False
4,93913,188172,9206492,16,2017-01-05 15:19:36.894,992,interrupt,iOS,17,17,wifi,False
5,93913,188172,9206492,16,2017-01-05 15:21:17.121,7,seek,iOS,17,17,wifi,False
6,93913,188172,9206492,16,2017-01-05 15:21:21.060,2,seek,iOS,17,17,wifi,False
7,93913,188172,9206492,16,2017-01-05 15:21:25.139,2,seek,iOS,17,17,wifi,False
8,93913,188172,9206492,16,2017-01-05 15:51:49.170,1817,seek,iOS,17,17,wifi,False
9,93913,188172,9206492,16,2017-01-05 15:51:53.570,3,seek,iOS,17,17,wifi,False


# Check Answer

In [11]:
answer = pd.read_csv('answer.csv')
answer

,user_id,time_slot_0,time_slot_1,time_slot_2,time_slot_3,time_slot_4,time_slot_5,time_slot_6,time_slot_7,time_slot_8,...,time_slot_18,time_slot_19,time_slot_20,time_slot_21,time_slot_22,time_slot_23,time_slot_24,time_slot_25,time_slot_26,time_slot_27
0,57159,0.31250,0.34375,0.21875,0.37500,0.40625,0.43750,0.34375,0.43750,0.31250,...,0.15625,0.12500,0.15625,0.18750,0.15625,0.21875,0.18750,0.28125,0.21875,0.34375
1,57160,0.03125,0.09375,0.03125,0.00000,0.06250,0.12500,0.03125,0.00000,0.12500,...,0.06250,0.00000,0.00000,0.06250,0.03125,0.00000,0.00000,0.09375,0.06250,0.00000
2,57161,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.03125,0.03125,0.00000,0.03125,0.00000,0.00000,0.00000,0.00000
3,57162,0.00000,0.00000,0.00000,0.00000,0.00000,0.03125,0.00000,0.00000,0.00000,...,0.15625,0.00000,0.03125,0.09375,0.15625,0.00000,0.03125,0.06250,0.03125,0.00000
4,57163,0.03125,0.03125,0.00000,0.00000,0.03125,0.03125,0.00000,0.00000,0.00000,...,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
5,57164,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,...,0.00000,0.03125,0.00000,0.00000,0.00000,0.03125,0.00000,0.03125,0.00000,0.00000
6,57165,0.03125,0.15625,0.06250,0.06250,0.00000,0.21875,0.09375,0.15625,0.00000,...,0.03125,0.00000,0.09375,0.06250,0.03125,0.03125,0.09375,0.12500,0.03125,0.09375
7,57166,0.00000,0.06250,0.00000,0.00000,0.00000,0.06250,0.00000,0.00000,0.00000,...,0.03125,0.00000,0.00000,0.06250,0.00000,0.00000,0.03125,0.12500,0.03125,0.00000
8,57167,0.43750,0.46875,0.21875,0.00000,0.65625,0.71875,0.15625,0.00000,0.59375,...,0.18750,0.00000,0.31250,0.46875,0.18750,0.03125,0.37500,0.50000,0.18750,0.00000
9,57168,0.65625,0.65625,0.00000,0.75000,0.75000,0.78125,0.00000,0.90625,0.87500,...,0.00000,0.12500,0.31250,0.21875,0.00000,0.12500,0.21875,0.09375,0.00000,0.65625
